In [1]:
### Import Packages ###
import os
import ast
import numpy as np
import math as math
import pandas as pd
import random as random

### Local Packages ###
from utils.Prediction import *
from utils.Selector import *
from utils.Auxiliary import *
from utils.Main import *

In [2]:
### Get Directory ###
cwd = os.getcwd()
ParentDirectory = os.path.abspath(os.path.join(cwd, ".."))

### DataType ###
DataTypeInput = "BostonHousing"

### Parameter Vector ###
ParameterVector = pd.read_csv(os.path.join(ParentDirectory, "Data", "ParameterVectors", "ParameterVector" + DataTypeInput + ".csv"))

# Simulation

In [ ]:
### Set Up ###
ErrorVecSimulation = []
HistoryVecSimulation = []

### Run Code ###
# for i in range(0,len(ParameterVector)):
for i in range(1,2):

    ### Set Up ###
    print("ParameterVector Simulation Number: ", i)

    ### Parameter Vector ###
    SimulationConfigInput = {"DataFileInput": ParameterVector.iloc[i]["Data"],
                            "Seed": int(ParameterVector.iloc[i]["Seed"]),
                            "TestProportion": float(ParameterVector.iloc[i]["TestProportion"]),
                            "CandidateProportion": float(ParameterVector.iloc[i]["CandidateProportion"]),
                            "SelectorType": str(ParameterVector.iloc[i]["SelectorType"]), 
                            "ModelType": str(ParameterVector.iloc[i]["ModelType"]), 
                            "UniqueErrorsInput": int(ParameterVector.iloc[i]["UniqueErrorsInput"]),
                            "n_estimators":int(ParameterVector.iloc[i]["n_estimators"]),
                            "regularization":float(ParameterVector.iloc[i]["regularization"]),
                            "rashomon_bound_adder":float(ParameterVector.iloc[i]["rashomon_bound_adder"]),
                            "Type":ParameterVector.iloc[i]["Type"]
                            }

    ### Sequential Learning Process ###
    SimulationResults = OneIterationFunction(SimulationConfigInput)

    ### Append ###
    ErrorVecSimulation.append(SimulationResults["ErrorVec"])
    HistoryVecSimulation.append(SimulationResults["SelectionHistory"])

ParameterVector Simulation Number:  1
Iteration: 0
null
Finding Optimal Objective...
treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 271
{
  "false": {
    "false": {
      "complexity": 0.009999999776482582,
      "loss": 0.012500000186264515,
      "name": "Y",
      "prediction": "False"
    },
    "feature": 8,
    "name": "RM_3",
    "reference": 1.0,
    "relation": "==",
    "true": {
      "false": {
        "false": {
          "complexity": 0.009999999776482582,
          "loss": 0.0,
          "name": "Y",
          "prediction": "False"
        },
        "feature": 5,
        "name": "NOX_3",
        "reference": 1.0,
        "relation": "==",
        "true": {
          "complexity": 0.009999999776482582,
          "loss": 0.0,
          "name": "Y",
          "prediction": "True"
        },
        "type": "rational"
      },
      "feature": 12,
      "name": "LSTAT_1",
      "reference": 1.0,
      "relation": "==",
    